In [40]:
# Imports
import torch
from torch.distributions import Normal,Uniform,Gamma,Laplace,OneHotCategorical
import os
import time
from functools import partial

from causal_cocycle.kernels import *
from causal_cocycle.kde import *
from causal_cocycle.regression_functionals import *
from causal_cocycle.distribution_estimation import *
from causal_cocycle.helper_functions import propensity_score

import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICSCORE_LIB_OK"]="TRUE"

In [14]:
# DGP functions
def cdf(X,t):
    return ((X<= t.T)*1).float().mean(0)

class IG:
    
    def __init__(self,alpha,beta):
        self.alpha = alpha
        self.beta = beta
        
    def sample(self,size):
        return 1/Gamma(self.alpha,self.beta).sample(size)

class Mixture1D:
    
    def __init__(self,base_dists,probabilities,noints,scales):
        self.dists = base_dists
        self.probabilities = probabilities
        self.noints = noints
        self.scales = scales
        
    def sample(self,size):
        C = OneHotCategorical(probabilities).sample(size)[:,0]
        Z = torch.zeros((size[0],len(probabilities)))
        for i in range(len(self.dists)):
            Z[:,i] = self.noints[i]+self.scales[i]*self.dists[i].sample(size).T
        return (Z*C).sum(1)[:,None]          

def policy(V, flip_prob = 0.00):
    Z = (V.mean(1)*len(V.T)**0.5)[:,None]
    X_correct =  (Z<-1)*0+(Z>=-1)*(Z<1)*1 + (Z>=1)*2
    flips = (Uniform(0,1).sample((len(V),1))<flip_prob)*1
    return X_correct*(1-flips) + torch.randint(3, (len(V),1))*flips

def new_policy(V, flip_prob = 0.00):
    Z = (V.mean(1)*len(V.T)**0.5)[:,None]
    X_correct =  (Z<-1)*0+(Z>=-1)*1
    flips = (Uniform(0,1).sample((len(V),1))<flip_prob)*1
    return X_correct*(1-flips) + torch.randint(2, (len(V),1))*flips

def shift(V,policy,coeffs):
    t = policy(V)
    z = V @ coeffs
    return 1/(1+torch.exp(z)) + ((t==0)*torch.exp(-0.1*(z+3)**2) + 
                                 (t==1)*torch.exp(-0.1*(z-0)**2)*0.75 + 
                                 (t==2)*torch.exp(-0.1*(z-3)**2)*0.5)

def scale(V,coeffs):
    z = V @ coeffs
    return 0.1*(torch.exp(-1/10*(z+2)**2*(z-2)**2)+1)

def DGP(N,D,policy,covariate_corr = 0, 
        covariate_dist = Normal(0,1),
        noise_dist = Normal(0,1)):
    Sigma = (1-covariate_corr)*torch.eye(D)+covariate_corr*torch.ones((D,D))
    A = torch.linalg.cholesky(Sigma)
    Z = covariate_dist.sample((N,D)) @ A.T
    U = noise_dist.sample((N,1))
    Y = shift(Z,policy,coeffs) + scale(Z,coeffs)*U
    X = torch.column_stack((policy(Z),Z))
    return Z,X,Y

In [57]:
# DGP set up
N = 10**4
D = 10
Zcorr = 0.0
flip_prob = 0.05
coeffs = 1/torch.linspace(1,D,D)[:,None]**1
coeffs *= 1/coeffs.sum()
means = torch.tensor([[-2, 0]]).T # means for mixture U dist
scales = torch.tensor([[-1.0, 1.0]]).T  # variances for mixture U dist
probabilities = torch.tensor([1/2,1/2]) # mixture probs for mixture U dist
base_dists = [IG(10,10),IG(1,1)]
noise_dist = Mixture1D(base_dists,probabilities,means,scales)
Zdist = Normal(0,1.5)
feature = lambda x,t: (torch.sigmoid(x)<=t.T).float()

In [58]:
# Method + opt set up
train_val_split = 0.5
functional = KRR_functional
kernel = gaussian_kernel
subsample = True
subsamples = 1000
lr = 0.025
nfold = 3
tol = 1e-3
ls_method = "med_heuristic"
hyper_grid = 2**torch.linspace(-10,0,10)
t_train = torch.linspace(0,1,100)[:,None]

In [59]:
# DGP
ntrain = int(train_val_split*N)
Z,X,Y = DGP(N,D,partial(policy,flip_prob = flip_prob),Zcorr,Zdist,noise_dist)
Ztrain,Xtrain,Ytrain = Z[:ntrain],X[:ntrain],Y[:ntrain]
Ztest,Xtest,Ytest = Z[ntrain:],X[ntrain:],Y[ntrain:]

In [75]:
# Getting data to train on
inputs_train = Xtrain
outputs_train = feature(Ytrain,t_train).float()

# Getting median heuristic
Distances= torch.tril((inputs_train[...,None]-inputs_train[...,None].T)**2)
ls = (Distances[Distances!=0].median()/2).sqrt() * torch.ones(D+1)
                      
# Defining model
k_x = kernel(lengthscale = torch.tensor(ls,requires_grad = True))
regressor = functional(k_x)
CE = Conditional_Expectation_Regressor(regressor)

# If not med heuristic gradient-descent on hypers
if ls_method != "med_heuristic":
    losses = CE.optimise(inputs_train,outputs_train,
                    learn_rate = lr, 
                    subsample = subsample,
                    subsamples = subsamples,
                    nfold = nfold,
                    tol = tol)
# Or, as long as there is a hyper to opt, do cvgridsearch
else:
    if len(regressor.hyperparameters)>1:
        CE.CVgridsearch(inputs_train,outputs_train,
                        nfold = nfold, 
                        subsample = subsample,
                        subsamples = subsamples,
                        hyper_grid = hyper_grid)

/tmp/ipykernel_319708/925322502.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  k_x = kernel(lengthscale = torch.tensor(ls,requires_grad = True))


In [76]:
# Sampling from interventional distribution
ntest = 5000
nintsample = ntest
Zintdist = Normal(1,1.5)
Zshift,Xshift,Yshift = DGP(nintsample,D,partial(policy),Zcorr,Zintdist,noise_dist)
Zint,Xint,Yint = DGP(nintsample,D,partial(new_policy),Zcorr,Zintdist,noise_dist)
Zshift_train,Xshift_train = Zshift[:ntest],Xshift[:ntest]
Xint_train = Xint[:ntest]

In [82]:
# cdf values
t = torch.linspace(0,1,1000)[:,None]
#outputs_train = feature(Yshift,t).float()
#inputs_train = Xshift_train
inputs_train = Xtrain
outputs_train = feature(Ytrain,t).float()

# Getting CDF
kde_cdf_int = CE.forward(outputs_train,inputs_train,Xint_train).mean(0).detach()
kde_cdf_shift = CE.forward(outputs_train,inputs_train,Xshift_train).mean(0).detach()

# True cdf
true_cdf_int = feature(Yint,t).mean(0)
true_cdf_shift = feature(Yshift,t).mean(0)

In [83]:
(kde_cdf_int-true_cdf_int).abs().mean()

tensor(0.0416)

In [26]:
# Training propensity score models
Propensity_score_model_est = []
Propensity_score_model_policy = []
Propensity_score_model_new_policy = []

# Estimating mistae probabilities
Xtrue = policy(Ztrain)
states = torch.unique(X[:,0]).int()
nstate = len(states)
P = torch.zeros((nstate,nstate))
for i in range(nstate):
    for j in range(nstate):
        P[i,j] = ((Xtrain[:,0]==states[i])*(Xtrue[:,0]==states[j])).float().sum()
P *= 1/P.sum(0)

propensity_model_est = propensity_score(P,policy)
propensity_model_new_policy = propensity_score(torch.eye(len(P)),new_policy)  
propensity_model_policy = propensity_score(torch.eye(len(P)),policy)  

In [27]:
# Training density models
kde_learn_rate = 0.1
kde_miniter = 100
kde_maxiter = 100
kde_tol = 1e-2
kde_nfold = 3
kde_reg = 1e-6

Densities_Z = []
Densities_Z_shift = []
    
    
kernel = inverse_gaussian_kernel(lengthscale = torch.ones(D),scale = 1.0)
density_z = KDE(kernel)
losses = density_z.optimise(Ztrain,kde_learn_rate,kde_miniter,kde_maxiter,kde_tol,kde_nfold,kde_reg)

kernel_shift = inverse_gaussian_kernel(lengthscale = torch.ones(D),scale = 1.0)
density_zshift = KDE(kernel_shift)
losses_shift = density_zshift.optimise(Zshift_train,kde_learn_rate,kde_miniter,kde_maxiter,kde_tol,kde_nfold,kde_reg)

iter 0 , loss =  tensor(68978.4219)
iter 10 , loss =  tensor(68913.3906)
iter 20 , loss =  tensor(68898.0156)
iter 30 , loss =  tensor(68892.9688)
iter 40 , loss =  tensor(68891.3359)
iter 50 , loss =  tensor(68890.8906)
iter 60 , loss =  tensor(68890.8516)
iter 70 , loss =  tensor(68890.8281)
iter 80 , loss =  tensor(68890.7969)
iter 90 , loss =  tensor(68890.7812)
iter 0 , loss =  tensor(68978.7656)
iter 10 , loss =  tensor(68914.7656)
iter 20 , loss =  tensor(68904.9766)
iter 30 , loss =  tensor(68901.3984)
iter 40 , loss =  tensor(68900.2422)
iter 50 , loss =  tensor(68899.7188)
iter 60 , loss =  tensor(68899.5625)
iter 70 , loss =  tensor(68899.5312)
iter 80 , loss =  tensor(68899.5391)
iter 90 , loss =  tensor(68899.5391)


In [ ]:
# True densities
class normal:

    def __init__(self,mu,sigma2):
        self.mu = mu
        self.var = sigma2

    def forward(self,data,X):
        return Normal(self.mu,self.var).log_prob(data).sum(1).exp()

#density_zshift = normal(1,1.5)
#density_z = normal(0,1.5)

In [28]:
# Getting IPW estimator
weights_shift = ((propensity_model_policy(Xtest,Ztest)*
                density_zshift.forward(Ztest,Zshift_train))/
                (propensity_model_est(Xtest,Ztest)*
                density_z.forward(Ztest,Ztrain))).detach()

weights_int = ((propensity_model_new_policy(Xtest,Ztest)*
                density_zshift.forward(Ztest,Zshift_train))/
                (propensity_model_est(Xtest,Ztest)*
                density_z.forward(Ztest,Ztrain))).detach()

weights_shift *= len(weights_shift)/weights_shift.sum()
weights_int *= len(weights_shift)/weights_int.sum()

IPW_cdf_shift = (weights_shift[:,None]*feature(Ytest,t)).mean(0)
IPW_cdf_int = (weights_int[:,None]*feature(Ytest,t)).mean(0)

In [29]:
# Getting DR estimator (start by adding on IPW term to outcome model
kde_DR_cdf_shift = kde_cdf_shift + IPW_cdf_shift
kde_DR_cdf_int = kde_cdf_int + IPW_cdf_int

# Updating DR estimator
kde_DR_cdf_shift -= (weights_shift[:,None]*CE.forward(outputs,inputs_train,Xtest)).mean(0)
kde_DR_cdf_int-= (weights_int[:,None]*CE.forward(outputs,inputs_train,Xtest)).mean(0)

In [30]:
print((kde_cdf_shift -true_cdf_shift).abs().mean())
print((kde_DR_cdf_shift -true_cdf_shift).abs().mean())

print((kde_cdf_int -true_cdf_int).abs().mean())
print((kde_DR_cdf_int -true_cdf_int).abs().mean())

tensor(6.3319e-07)
tensor(5.9968e-06, grad_fn=<MeanBackward0>)
tensor(3.1849e-06)
tensor(4.5334e-06, grad_fn=<MeanBackward0>)
